In [1]:
import pandas as pd

In [50]:
all_items = pd.read_csv('https://download.bls.gov/pub/time.series/cu/cu.data.1.AllItems', sep='\t')
all_items = all_items.applymap(lambda x: x.strip() if isinstance(x, str) else x)
all_items.columns = [x.strip() for x in all_items.columns]

series_descriptions = pd.read_csv('https://download.bls.gov/pub/time.series/cu/cu.series', sep='\t')
series_descriptions = series_descriptions.applymap(lambda x: x.strip() if isinstance(x, str) else x)
series_descriptions.columns = [x.strip() for x in series_descriptions.columns]

period_description = pd.read_csv('https://download.bls.gov/pub/time.series/cu/cu.period', sep='\t')
period_description = period_description.applymap(lambda x: x.strip() if isinstance(x, str) else x)
period_description.columns = [x.strip() for x in period_description.columns]

periodicity_description = pd.read_csv('https://download.bls.gov/pub/time.series/cu/cu.periodicity', sep='\t')
periodicity_description = periodicity_description.applymap(lambda x: x.strip() if isinstance(x, str) else x)
periodicity_description.columns = [x.strip() for x in periodicity_description.columns]

seasonal_description = pd.read_csv('https://download.bls.gov/pub/time.series/cu/cu.seasonal', sep='\t')
seasonal_description = seasonal_description.applymap(lambda x: x.strip() if isinstance(x, str) else x)
seasonal_description.columns = [x.strip() for x in seasonal_description.columns]

In [128]:
#enhance timeseries data with meta data located in other files.
all_items_enhanced = all_items.merge(series_descriptions, how='left',  left_on='series_id', right_on='series_id')
all_items_enhanced = all_items_enhanced.merge(period_description, how='left',  left_on='period', right_on='period')
all_items_enhanced = all_items_enhanced.merge(periodicity_description, how='left',  left_on='periodicity_code', right_on='periodicity_code')
all_items_enhanced = all_items_enhanced.merge(seasonal_description, how='left',  left_on='seasonal', right_on='seasonal_code')

In [130]:
# 1. filter to only the monthly data. 
# 2. add a datetime column
# 3. sort rows by dates. 
# 4. set multi-index to series_id and date for easy access. 

periods_wanted = [ 'January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December' ]
all_items_enhanced = all_items_enhanced[ all_items_enhanced.period_name.isin(periods_wanted)].reset_index()
all_items_enhanced['date'] = pd.to_datetime(all_items_enhanced['period'].str[-2:] + '/' +  all_items_enhanced['year'].astype(str), format='%m/%Y') + pd.offsets.MonthEnd(0)
all_items_enhanced = all_items_enhanced.sort_values('date')
all_items_enhanced = all_items_enhanced.set_index(['series_id', 'date'])

In [ ]:
all_items_enhanced

In [125]:
all_items_enhanced.xs('CUUR0000SA0', level='series_id')['value'].pct_change(12)

date
1913-01-31         NaN
1913-02-28         NaN
1913-03-31         NaN
1913-04-30         NaN
1913-05-31         NaN
                ...   
2021-10-31    0.062219
2021-11-30    0.068090
2021-12-31    0.070364
2022-01-31    0.074799
2022-02-28    0.078711
Name: value, Length: 1310, dtype: float64

In [126]:
all_items_enhanced.xs('CUSR0000SA0', level='series_id')['value'].pct_change(1)

date
1947-01-31         NaN
1947-02-28    0.006518
1947-03-31    0.017576
1947-04-30    0.000000
1947-05-31   -0.002273
                ...   
2021-10-31    0.008665
2021-11-30    0.006992
2021-12-31    0.005752
2022-01-31    0.006451
2022-02-28    0.007977
Name: value, Length: 902, dtype: float64